<a href="https://colab.research.google.com/github/RohanKatyayani/finrisk-copilot/blob/main/llama_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing dependencies for LLaMA + LoRA
!pip install -q transformers accelerate bitsandbytes peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving german_credit_explanations.jsonl to german_credit_explanations.jsonl


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="german_credit_explanations.jsonl")

# Preview
print(dataset)
print(dataset["train"][0])

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'label', 'explanation'],
        num_rows: 1000
    })
})
{'input': 'status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67', 'label': 'bad', 'explanation': 'Application denied due to stable financial profile.'}


In [ ]:
from datasets import load_dataset

# Loading the uploaded dataset
dataset = load_dataset("json", data_files="german_credit_explanations.jsonl")

# Preview
print(dataset)
print(dataset['train'][0])  # Show first sample

DatasetDict({
    train: Dataset({
        features: ['input', 'label', 'explanation'],
        num_rows: 1000
    })
})
{'input': 'status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67', 'label': 'bad', 'explanation': 'Application denied due to stable financial profile.'}


Example format for training

### Instruction:
You are a bank risk officer. Given customer credit information, decide if the application is good or bad and explain why.

### Input:
status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67

### Response:
Label: bad
Explanation: Application denied due to stable financial profile.

In [ ]:
def format_example(example):
    return {
        "text": f"""### Instruction:
You are a bank risk officer. Given customer credit information, decide if the application is good or bad and explain why.

### Input:
{example['input']}

### Response:
Label: {example['label']}
Explanation: {example['explanation']}"""
    }

# Apply formatting
dataset = dataset.map(format_example)

# Check one example
print(dataset['train'][0]['text'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Instruction:
You are a bank risk officer. Given customer credit information, decide if the application is good or bad and explain why.

### Input:
status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67

### Response:
Label: bad
Explanation: Application denied due to stable financial profile.


In [ ]:
!git clone https://github.com/RohanKatyayani/finrisk-copilot.git
%cd finrisk-copilot

Cloning into 'finrisk-copilot'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 92 (delta 31), reused 70 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 280.48 KiB | 1.91 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/finrisk-copilot


In [ ]:
!pip install -q datasets transformers accelerate

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data/credit_instructions.jsonl")

print(dataset)
print(dataset["train"][0])  # show first sample

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1000
    })
})
{'instruction': 'Explain the credit risk decision for the following applicant profile.', 'input': 'status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67', 'output': 'Decision: bad. Explanation: Application denied due to stable financial profile.'}


In [ ]:
# Split into train and validation (90% train / 10% validation)
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

print(dataset)
print(f"Train size: {len(dataset['train'])}")
print(f"Validation size: {len(dataset['test'])}")

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 900
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 100
    })
})
Train size: 900
Validation size: 100


In [ ]:
# Step 2: Train/Validation split
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

print(dataset)

print(f"Train size: {len(dataset['train'])}")
print(f"Validation size: {len(dataset['test'])}")

# Peek at a few samples
print("\nSample from train:")
print(dataset["train"][0])

print("\nSample from validation:")
print(dataset["test"][0])

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 810
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 90
    })
})
Train size: 810
Validation size: 90

Sample from train:
{'instruction': 'Explain the credit risk decision for the following applicant profile.', 'input': 'status=no checking account, duration=36, savings=... < 100 DM, employment=1 <= ... < 4 years, amount=3349, age=28', 'output': 'Decision: good. Explanation: Application approved due to stable financial profile.'}

Sample from validation:
{'instruction': 'Explain the credit risk decision for the following applicant profile.', 'input': 'status=... < 100 DM, duration=12, savings=... < 100 DM, employment=1 <= ... < 4 years, amount=1620, age=30', 'output': 'Decision: bad. Explanation: Application denied due to stable financial profile.'}


In [2]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from google.colab import files

uploaded = files.upload()

Saving german_credit_explanations.jsonl to german_credit_explanations.jsonl


In [9]:
model.push_to_hub("rohankatyayani/tinyllama-credit-explainer")
tokenizer.push_to_hub("rohankatyayani/tinyllama-credit-explainer")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...c1goqnw/model.safetensors:   1%|          | 25.1MB / 4.40GB            

No files have been modified since last commit. Skipping to prevent empty commit.


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...p169745q7/tokenizer.model: 100%|##########|  500kB /  500kB            

CommitInfo(commit_url='https://huggingface.co/rohankatyayani/tinyllama-credit-explainer/commit/16a412394d99d6b6f628af60cba7200dce83c53c', commit_message='Upload tokenizer', commit_description='', oid='16a412394d99d6b6f628af60cba7200dce83c53c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rohankatyayani/tinyllama-credit-explainer', endpoint='https://huggingface.co', repo_type='model', repo_id='rohankatyayani/tinyllama-credit-explainer'), pr_revision=None, pr_num=None)

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load from Hugging Face Hub
model_id = "rohankatyayani/tinyllama-credit-explainer"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Wrap into a pipeline
credit_explainer = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)

# Test prompt
prompt = "status=... < 100 DM, duration=12, savings=... < 100 DM, employment=1 <= ... < 4 years, amount=4870, age=53. Explain the decision in a bank-friendly tone:"
result = credit_explainer(prompt)

print(result[0]["generated_text"])

tokenizer_config.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/410 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


status=... < 100 DM, duration=12, savings=... < 100 DM, employment=1 <= ... < 4 years, amount=4870, age=53. Explain the decision in a bank-friendly tone: “We have decided to provide this individual with a line of credit, based on the information provided. The line of credit has a $1000 credit limit, is for 12 months, and will be set at 15% APR. In order to qualify for this line of credit, the individual must maintain a satisfactory credit score, with a minimum credit utilization of 30%. The application and approval process will take approximately 24 business hours from
